In [1]:
#import library
import cassandra


In [2]:
#connect to cluster
from cassandra.cluster import Cluster
try:
    cluster=Cluster(['127.0.0.1'])
    session =cluster.connect()
except Exception as e:
    print(e)
    

In [3]:
## create a keyspace and connect

try:
    session.execute("""
    create keyspace if not exists udacity
    with replication =
    {'class':'SimpleStrategy','replication_factor':'1'}
    """)
except Exception as e:
    print(e)
    
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Let's imagine we would like to start creating a new Music Library of albums. 
### We want to ask 4 question of our data
#### 1. Give me every album in my music library that was released in a 1965 year
#### 2. Give me the album that is in my music library that was released in 1965 by "The Beatles"
#### 3. Give me all the albums released in a given year that was made in London 
#### 4. Give me the city that the album "Rubber Soul" was recorded

In [4]:
query="create table if not exists music_library"
query=query+"(year int, city text, artist_name text, album_name text, primary key (year, artist_name, album_name))"
try:
    session.execute(query)
except Exception as e:
    print(e)
    

In [5]:
### Insert to tables
query="insert into music_library(year, artist_name, album_name, city)"
query=query+"values(%s,%s,%s,%s)"
try:
    session.execute(query, (1970, "The Beatles", "Let it Be", "Liverpool"))
except Exception as e:
    print(e)
    
try:
    session.execute(query, (1965, "The Beatles", "Rubber Soul", "Oxford"))
except Exception as e:
    print(e)
    
try:
    session.execute(query, (1965, "The Who", "My Generation", "London"))
except Exception as e:
    print(e)

try:
    session.execute(query, (1966, "The Monkees", "The Monkees", "Los Angeles"))
except Exception as e:
    print(e)

try:
    session.execute(query, (1970, "The Carpenters", "Close To You", "San Diego"))
except Exception as e:
    print(e)

In [8]:
#Q1: Give me every album in my music library that was released in a 1965 year
query="select * from music_library where year=1965"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.year,row.artist_name,row.album_name,row.city)
    

1965 The Beatles Rubber Soul Oxford
1965 The Who My Generation London


In [10]:
#Q2: Give me the album that is in my music library that was released in 1965 by "The Beatles"
query="select * from music_library where year=1965 and artist_name='The Beatles'"
try:
    rows=session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.year,row.artist_name,row.album_name,row.city)
    

1965 The Beatles Rubber Soul Oxford


In [11]:
#Q3: Give me all the albums released in a given year that was made in London
query="select * from music_library where location = 'London'"
try:
    rows=session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.year,row.artist_name,row.album_name,row.city)
    

Error from server: code=2200 [Invalid query] message="Undefined column name location"


#### We can not try to access a column or a clustering column if we have not used the other defined clustering column
, Then we have a different way:

In [13]:
#Q4: Give me the city that the album "Rubber Soul" was recorde
query="select city from music_library where year=1965 and artist_name='The Beatles' and album_name='Rubber Soul'"
try:
    rows=session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.city)
    

Oxford


In [14]:
#drop table:
try:
    session.execute("drop table music_library")
except Exception as e:
    print(e)

In [ ]:
### shutd